In [1]:
import pandas as pd
import numpy as np
import shapefile #pip install pyshp
from geopy.distance import vincenty
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
import glob
import os

In [2]:
path = 'D:\\Users\\Qoo\\Desktop\\taipower\\data\\'

In [3]:
rain = pd.read_csv(path + 'rain.csv',encoding='big5')
df_1107 = pd.read_csv(path+ 'df_1107.csv')

In [5]:
rain.columns

Index([      u'Unnamed: 0',    u'accu_end_time',       u'accu_value',
                    u'num',             u'stno',          u'stntype',
       u'typhoon_cht_name',     u'typhoon_name',          u'typhoon',
                    u'day',               u'ts',         u'pre_avg3',
              u'pre_accu3',        u'pre_accu6',       u'pre_accu12',
               u'pre_max3',         u'pre_max6',        u'pre_max12',
              u'pre_mean3',        u'pre_mean6',       u'pre_mean12'],
      dtype='object')

In [ ]:
#rain.iloc[0:10,[1,2,3,4,8,11,12,13,14,15,16,17,18,19,20]]

In [9]:
#平均3小時雨量
mean_rain_3hr = rain.groupby(['stno','typhoon'],as_index=False).agg({u'pre_accu3':'mean'})
mean_rain_3hr.columns.values[2] = 'mean_accu_3hour_rain'
#平均6小時雨量
mean_rain_6hr = rain.groupby(['stno','typhoon'],as_index=False).agg({u'pre_accu6':'mean'})
mean_rain_6hr.columns.values[2] = 'mean_accu_6hour_rain'
#平均12小時雨量
mean_rain_12hr = rain.groupby(['stno','typhoon'],as_index=False).agg({u'pre_accu12':'mean'})
mean_rain_12hr.columns.values[2] = 'mean_accu_12hour_rain'
#最大3小時雨量
max_rain_3hr = rain.groupby(['stno','typhoon'],as_index=False).agg({u'pre_accu3':'max'})
max_rain_3hr.columns.values[2] = 'max_accu_3hour_rain'
#最大6小時雨量
max_rain_6hr = rain.groupby(['stno','typhoon'],as_index=False).agg({u'pre_accu6':'max'})
max_rain_6hr.columns.values[2] = 'max_accu_6hour_rain'
#最大12小時雨量
max_rain_12hr = rain.groupby(['stno','typhoon'],as_index=False).agg({u'pre_accu12':'max'})
max_rain_12hr.columns.values[2] = 'max_accu_12hour_rain'

#達到幾次豪雨指標(三小時累積降雨 100MM以上)
how_rain_times2 = rain.loc[rain.pre_accu3 >= 100,:].groupby(['stno','typhoon'],as_index=False).agg({u'pre_accu3':'count'})
how_rain_times2.columns.values[2] = 'how_rain_count_rule2'

In [10]:
df_1107 = pd.merge(df_1107, mean_rain_3hr, how='left', on=['stno','typhoon'])
df_1107 = pd.merge(df_1107, mean_rain_6hr, how='left', on=['stno','typhoon'])
df_1107 = pd.merge(df_1107, mean_rain_12hr, how='left', on=['stno','typhoon'])
df_1107 = pd.merge(df_1107, max_rain_3hr, how='left', on=['stno','typhoon'])
df_1107 = pd.merge(df_1107, max_rain_6hr, how='left', on=['stno','typhoon'])
df_1107 = pd.merge(df_1107, max_rain_12hr, how='left', on=['stno','typhoon'])
df_1107 = pd.merge(df_1107, how_rain_times2, how='left', on=['stno','typhoon'])

In [13]:
print len(df_1107.columns.values)

92


In [16]:
#補值
df_1107v2 = pd.concat([ df_1107.iloc[:,0:85], df_1107.iloc[:,85:].fillna(0)],axis=1)

In [17]:
df_1107v2.to_csv(path+'df_1107v2.csv')